# Process HPWREN Camera Metadata (Firemap API)

<b>Summary:</b><br>
Processes the hpwren only camera metadata pulled from firemap pylaski api.

- Read in raw camera metadata
- Filter for HPWREN cameras only
- Transform
    - Extract direction as col
    - Extract image name as col
    - Extract lat, long, elevation as cols
    - Merge with manually entered camera metadata
    1. If manual data missing camera_name then merge (append/correct data)
    2. If manual data has camera_name then union (add missing data)
- Save processed hpwren camera metadata file

<b>Output:</b><br>
.<br>
└── data<br>
&emsp;&emsp;&emsp;└── processed<br>
&emsp;&emsp;&emsp;&nbsp;&nbsp;&emsp;&emsp;&nbsp;└── camera_metadata_hpwren.csv<br>

<b>Areas for Improvement:</b><br>

## Read All Camera Metadata

In [1]:
import json
from ast import literal_eval
from datetime import datetime

import pandas as pd
import pytz

In [2]:
# literal_eval to read columns as lists
cameras_df = pd.read_csv(
    "../../data/raw/camera_metadata.csv",
    converters={
        "geometry.coordinates": literal_eval,
        "properties.latest-images": literal_eval,
    },
)

In [3]:
cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.is_patrol_mode,properties.description.lastupdate,properties.description.region,properties.description.fov,properties.description.activated_at,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Camera breakdown
# all camera metadata either has hpwren or axis prefix
cameras_df["properties.description.id"].str.split(r"_|-", expand=True)[0].str.extract(
    r"([a-zA-Z]+)"
)[0].str.lower().value_counts()

axis      1055
hpwren     103
Name: 0, dtype: int64

## Filter for HPWREN cameras only

In [5]:
# just get hpwren cameras (ignore axis)

hpwren_cameras_df = cameras_df[
    cameras_df["properties.description.id"]
    .str.lower()
    .str.contains("hpwren", regex=False)
].copy()

# dictoinary example below:
# hpwren_cameras = []
# for feat in features:
#     feat_id = feat["properties"]["description"]["id"]
#     if "hpwren" in feat_id.lower():
#         # print(feat_id)
#         hpwren_cameras.append(feat)

In [6]:
len(hpwren_cameras_df)

103

## Transformations

### Extract direction

In [7]:
# Extract direction from description
hpwren_cameras_df["direction"] = hpwren_cameras_df[
    "properties.description.id"
].str.split("_", n=1, expand=True)[1]

In [8]:
hpwren_cameras_df["direction"].value_counts()

north                26
east                 26
south                25
west                 25
unknown direction     1
Name: direction, dtype: int64

### Extract image name

In [9]:
# image name
hpwren_cameras_df["image_id"] = (
    hpwren_cameras_df["properties.latest-images"]
    .str[0]
    .str[0]
    .str["image"]
    .str.split("/")
    .str[-1]
    .str.split(".")
    .str[0]
)

In [10]:
hpwren_cameras_df.shape

(103, 31)

In [11]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.region,properties.description.fov,properties.description.activated_at,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current,direction,image_id
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c


### Extract long, lat, elevation

In [12]:
hpwren_cameras_df["long"] = hpwren_cameras_df["geometry.coordinates"].str[0]
hpwren_cameras_df["lat"] = hpwren_cameras_df["geometry.coordinates"].str[1]
hpwren_cameras_df["elevation"] = hpwren_cameras_df["geometry.coordinates"].str[2]

In [13]:
hpwren_cameras_df.shape

(103, 34)

In [14]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,properties.description.is_currently_patrolling,properties.description.az_current,properties.description.state,properties.description.fov_rt,properties.description.tilt_current,direction,image_id,long,lat,elevation
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,unknown direction,tje-1-mobo-c,-117.1200,32.5500,10
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,north,bm-n-mobo-c,-116.8081,33.1599,4055
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,east,bm-e-mobo-c,-116.8081,33.1599,4055
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,south,bm-s-mobo-c,-116.8081,33.1599,4055
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,west,bm-w-mobo-c,-116.8081,33.1599,4055


### Merge with manually entered camera metadata

- If manual data missing camera_name then merge
- If manual data has camera_name then union

In [15]:
cam_man_df = pd.read_csv("../../data/raw/camera_metadata_manual.csv")
cam_man_df = cam_man_df.add_suffix("_manual")

In [16]:
cam_man_df.shape

(138, 12)

In [17]:
cam_man_df.head()

,camera_id_manual,image_id_manual,prev_image_ids_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual,center_lat_manual,center_long_manual
0,hpwren0_unknown direction,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hpwren1_north,NaN,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.181599,-116.807554
2,hpwren1_east,NaN,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.158781,-116.790230
3,hpwren1_south,NaN,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.157932,-116.807962
4,hpwren1_west,NaN,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.159091,-116.858706


In [18]:
cam_man_na_df = cam_man_df[cam_man_df["camera_name_manual"].isna()]

In [19]:
cam_man_na_df.shape

(103, 12)

In [20]:
hpwren_cameras_df = hpwren_cameras_df.merge(
    cam_man_na_df,
    left_on="properties.description.id",
    right_on="camera_id_manual",
    how="inner",
)

In [21]:
hpwren_cameras_df.shape

(103, 46)

In [22]:
cam_man_nona_df = cam_man_df[~cam_man_df["camera_name_manual"].isna()]

In [23]:
cam_man_nona_df.shape

(35, 12)

In [24]:
hpwren_cameras_df = pd.concat([hpwren_cameras_df, cam_man_nona_df])

In [25]:
hpwren_cameras_df.shape

(138, 46)

In [26]:
hpwren_cameras_df.head()

,type,geometry.type,geometry.coordinates,properties.description.name,properties.description.id,properties.description.url,properties.latest-images,properties.description.type,properties.description.ptz,properties.description.zoom_current,...,prev_image_ids_manual,camera_name_manual,direction_manual,gmap_lat_manual,gmap_long_manual,elevation_manual,x_resolution_manual,y_resolution_manual,center_lat_manual,center_long_manual
0,Feature,Point,"[-117.12, 32.55, 10]",NaN,hpwren0_unknown direction,http://hpwren.ucsd.edu/cameras/TJE.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_north,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.181599,-116.807554
2,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_east,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.158781,-116.790230
3,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_south,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.157932,-116.807962
4,Feature,Point,"[-116.8081, 33.1599, 4055]",Big Black Mountain,hpwren1_west,http://hpwren.ucsd.edu/cameras/BBlackMtn.html,[[{'image': 'http://hpwren.ucsd.edu/cameras/L/...,NaN,NaN,NaN,...,[],NaN,NaN,33.159927,-116.808092,NaN,3072.0,2048.0,33.159091,-116.858706


In [27]:
# If original value null, then fill with new value

hpwren_cameras_df["camera_id"] = hpwren_cameras_df["properties.description.id"].fillna(
    hpwren_cameras_df["camera_id_manual"]
)

hpwren_cameras_df["camera_name"] = hpwren_cameras_df[
    "properties.description.name"
].fillna(hpwren_cameras_df["camera_name_manual"])

hpwren_cameras_df["image_id"] = hpwren_cameras_df["image_id"].fillna(
    hpwren_cameras_df["image_id_manual"]
)

hpwren_cameras_df["direction"] = hpwren_cameras_df["direction"].fillna(
    hpwren_cameras_df["direction_manual"]
)

hpwren_cameras_df["elevation"] = hpwren_cameras_df["elevation"].fillna(
    hpwren_cameras_df["elevation_manual"]
)

# If manual value null, then fill with orig value

hpwren_cameras_df["lat"] = hpwren_cameras_df["gmap_lat_manual"].fillna(
    hpwren_cameras_df["lat"]
)

hpwren_cameras_df["long"] = hpwren_cameras_df["gmap_long_manual"].fillna(
    hpwren_cameras_df["long"]
)

In [28]:
columns = [
    "camera_id",
    "direction",
    "camera_name",
    "image_id",
    "prev_image_ids_manual",
    "long",
    "lat",
    "elevation",
    "geometry.type",
    "geometry.coordinates",
    "x_resolution_manual",
    "y_resolution_manual",
    "center_lat_manual",
    "center_long_manual",
    "properties.description.url",
    # "properties.latest-images",
]
hpwren_cameras_final_df = hpwren_cameras_df[columns]

hpwren_cameras_final_df = hpwren_cameras_final_df.rename(
    columns={
        "prev_image_ids_manual": "prev_image_ids",
        "x_resolution_manual": "x_resolution",
        "y_resolution_manual": "y_resolution",
        "center_lat_manual": "center_lat",
        "center_long_manual": "center_long",
    },
)

hpwren_cameras_final_df

,camera_id,direction,camera_name,image_id,prev_image_ids,long,lat,elevation,geometry.type,geometry.coordinates,x_resolution,y_resolution,center_lat,center_long,properties.description.url
0,hpwren0_unknown direction,unknown direction,NaN,tje-1-mobo-c,[],-117.120000,32.550000,10.0,Point,"[-117.12, 32.55, 10]",NaN,NaN,NaN,NaN,http://hpwren.ucsd.edu/cameras/TJE.html
1,hpwren1_north,north,Big Black Mountain,bm-n-mobo-c,[],-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.181599,-116.807554,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
2,hpwren1_east,east,Big Black Mountain,bm-e-mobo-c,[],-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.158781,-116.790230,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
3,hpwren1_south,south,Big Black Mountain,bm-s-mobo-c,[],-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.157932,-116.807962,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
4,hpwren1_west,west,Big Black Mountain,bm-w-mobo-c,[],-116.808092,33.159927,4055.0,Point,"[-116.8081, 33.1599, 4055]",3072.0,2048.0,33.159091,-116.858706,http://hpwren.ucsd.edu/cameras/BBlackMtn.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,hpwren_missing7_south,south,SMER TCS9 HPWREN,smer-tcs9-s-mobo-c,[],-117.530000,33.710000,5669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,hpwren_missing7_west,west,SMER TCS9 HPWREN,smer-tcs9-w-mobo-c,[smer-tcs9-mobo-c],-117.530000,33.710000,5669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,hpwren_missing8_west,east,SMER TCS3 HPWREN,smer-tcs3-mobo-c,[smer-tcs3-mobo-c],-117.180000,33.450000,1400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,hpwren_missing9_west,west,SMER TCS8 HPWREN,smer-tcs8-mobo-c,[smer-tcs8-mobo-c],-117.150000,33.460000,2063.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Write camera metadata to raw and processed folders

In [29]:
# current_time_str = datetime.now(tz=pytz.UTC).strftime("%Y%m%d_%H_%M_%S")

# hpwren cameras only
hpwren_cameras_final_df.to_csv(
    f"../../data/processed/camera_metadata_hpwren.csv", index=False
)